In [1]:
import os
import pandas as pd
import numpy as np
import awkward as ak
import uproot_methods
print(ak.__version__)

0.12.0


In [6]:
##both should be numpy array
ls = np.array([1,2,1])
mask = np.array([2,1])
a = ak.JaggedArray.fromcounts(mask, ls)

In [29]:
##and Professor suggests that we could use mass, classifacation for later application
def SetAKArr(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    n_particles_ls = []
    px_ls = []
    py_ls = []
    pz_ls = []
    energy_ls = []
    mass_ls = []
    _label1 = []
    _label2 = []
    n = 0
    for line in lines:
        if line.startswith('E'):
            if not n == 0:
                n_particles_ls.append(n)
                n = 0
            exp_inf = line.split()
            _label1.append(int(exp_inf[1]))
            _label2.append(1-int(exp_inf[1]))
        else:
            par = line.split()
            ##particle +1
            n = n + 1
            px_ls.append(float(par[1]))
            py_ls.append(float(par[2]))
            pz_ls.append(float(par[3]))
            energy_ls.append(float(par[4]))
            mass_ls.append(float(par[5]))
            
    px_arr = np.array(px_ls)
    py_arr = np.array(py_ls)
    pz_arr = np.array(pz_ls)
    energy_arr = np.array(energy_ls)
    mass_arr = np.array(mass_ls)
    n_particles = np.array(n_particles_ls)
#     print(n_particles)
    px = ak.JaggedArray.fromcounts(n_particles, px_arr)
    py = ak.JaggedArray.fromcounts(n_particles, py_arr)
    pz = ak.JaggedArray.fromcounts(n_particles, pz_arr)
    energy = ak.JaggedArray.fromcounts(n_particles, energy_arr)
    mass = ak.JaggedArray.fromcounts(n_particles, mass_arr)
    p4 = uproot_methods.TLorentzVectorArray.from_cartesian(px, py, pz, energy)
    ##Create an Order Dic
    from collections import OrderedDict
    v = OrderedDict()
    v['part_px'] = px
    print(px)
    v['part_py'] = py
    v['part_pz'] = pz
    v['part_energy'] = energy
    v['part_mass'] = mass
#     ls1 = [1,2,3,4]
#     ls2 = [5,6,7,8]
    v['label'] = np.stack((_label1, _label2), axis=-1)
#     print(v['label'])
    return v

In [30]:
x = SetAKArr("train.txt")

[[-0.3162697467459443 0.02881739128528447 0.21155536277901646 ... 0.12861184258278482 0.08675066284909895 0.24405372120757218] [0.40624541033679146 -0.026873773856092374 -0.24371108140810097 ... -0.07454940257199988 0.030960933939939032 0.02788891197534895] [-0.20377421531128534 0.0861862008640884 -0.01919939482246239 ... -0.1419353552313762 0.17758832302043198 0.03654769206730623] ... [0.09664022256634287 -0.5160845643206918 -0.13627603462763913 ... 0.25147100900218167 -0.5802942602256381 0.2087934788137815] [1.0692057320538 -0.2840704432494957 -0.28187264363362263 ... -0.200251634179255 0.06510378067370481 -0.501549279920893] [1.4233357442342003 2.4936343779468115e-06 -0.42773012493993784 ... 0.12816990510311949 -1.0414738530533012 -0.12094197911558303]]


In [2]:
myarray = ak.JaggedArray.fromiter([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
print(myarray[1:])
# returns <JaggedArray [[] [4.4 5.5]] at 7f02018afc18>
print(myarray[100:])
# returns <JaggedArray [] at 7f020c214438>

TypeError: a bytes-like object is required, not 'list'

In [4]:
myarray = ak.JaggedArray.fromiter([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
indexes = ak.JaggedArray.fromiter([[1, 2, 2, 0], [], [1]])
myarray[indexes]
# returns <JaggedArray [[3.3 3.3 1.1] [] [5.5]] at 7f02018afa58>

TypeError: a bytes-like object is required, not 'list'

In [14]:
myarray = ak.JaggedArray.fromcounts([3, 0, 2], ak.Table(
              x=[1, 2, 3, 4, 5],
              y=[1.1, 2.2, 3.3, 4.4, 5.5],
              z=[True, False, True, False, False]))
print(myarray["x"])
# returns <JaggedArray [[1 2 3] [] [4 5]] at 7f020e8122b0>
print(myarray[["x", "y"]])
# returns <JaggedArray [[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]] at 7f02018af860>
print(myarray[["x", "y"]].columns)
# returns ['x', 'y']
print(myarray.count())
print(myarray)
myarray["x"] = [1, 2, 3, 4, 5]
print(myarray["x"])

[[1 2 3] [] [4 5]]
[[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]]
['x', 'y']
[<Row 0> <Row 1> <Row 2>]
[[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]]
[[1 1 1] [] [3 3]]
